In [1]:
from sys import path as wd
from os import path, getcwd
from time import sleep
import pandas as pd

workdir = path.dirname(getcwd())
wd.append(workdir)

from dags.operadores import (GoogleSheetManager, GoogleDriveManager)
from dags.estruturas import (InformationManager)
from dags.sidrapi import (SidraManager)
from dags.configure_directory import (DirectoryManager)
import json

C:\Users\usuario\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


c:\Users\usuario\OneDrive\Documentos\R\Banco de Dados\Sidra-PI V2\Sidra-PI V2\data


In [2]:
data_json = path.join(workdir, "data", "__data__info.json")
local_dir = path.join(workdir, "data", "processed")
final_dir = path.join(workdir, "data", "formatted")

with open(data_json, 'r') as file:
    info_data = json.load(file)

#
credentials_dir = path.join(workdir, "data", "credentials.json")
folder_name = info_data["MainFolder"]["Name"]
knowledge_sheet_url = r'https://docs.google.com/spreadsheets/d/11-CI4nLXTte0OcDVMl-My4PtOdif7ziWB0GKg6hkJMw/edit?usp=sharing'#info_data["Knowledge base"]

In [3]:
# Nomeclaturas 
TAB_COLUMN_NAME = "tabela" # coluna com o nome da tabela
VAR_COLUMN_NAME = "Número da Variável"
CLASS_COLUMN_NAME = "Classificador da Tabela"
SPATIAL_COLUMN_NAME = "Níveis Territoriais"
PERIOD_COLUMN_NAME = "Tipo do Período"
TABLE_INDEX = 0

In [4]:
# Classes
sheet_manager = GoogleSheetManager(credentials_dir)
drive_manager = GoogleDriveManager(credentials_dir, data_json)
information_manager = InformationManager(knowledge_sheet_url)
sidra_manager = SidraManager()

In [5]:
# df_information = sheet_manager.get_data_info_from_url(knowledge_sheet_url)[[TAB_COLUMN_NAME, SPATIAL_COLUMN_NAME]]

In [6]:
# tabela de informações
data_info_dir = path.join(local_dir, "dados_sidra2.xlsx")
df_sidra_tables = pd.read_excel(data_info_dir, sheet_name="Tabelas")
df_sidra_vars = pd.read_excel(data_info_dir, sheet_name="Variáveis")

In [7]:
continuar_de = 6588
indice = df_sidra_tables[df_sidra_tables[TAB_COLUMN_NAME] == continuar_de].index[0]
print(indice)

# df_sidra_tables.loc[indice:, :].count()

64


In [8]:
for idx, row in df_sidra_tables.loc[indice:, :].iterrows():
    table_number = row[TAB_COLUMN_NAME]
    tb_var = df_sidra_vars[df_sidra_vars[TAB_COLUMN_NAME] == table_number]

    pages_data = []
    pages_names = []

    for i, row_var in tb_var.iterrows():
        try:
            df = sidra_manager.fetch_sidra_data(t=table_number,
                                                v=row_var[VAR_COLUMN_NAME],
                                                c=row[CLASS_COLUMN_NAME],
                                                n=row[SPATIAL_COLUMN_NAME],
                                                p=row[PERIOD_COLUMN_NAME])
            # print(df.head(3))
            if df is not None:
                pages_data.append(df)
                nome_aba = row_var["Descrição"].split("-")[0]
                nome_aba = nome_aba.translate({ord(c): None for c in '\\/*?:"[]'})
                nome_aba = nome_aba[:31]  # Limiting to 31 characters
                pages_names.append(nome_aba)
            else:
                print(f"No data returned for table {table_number}, variable {row_var[VAR_COLUMN_NAME]}")
            sleep(1)

        except Exception as e:
            print(f"An error occurred for table number {table_number}: {e}")
            sleep(10)
            continue

    if pages_data:
        with pd.ExcelWriter(f'{local_dir}\\Tabela {table_number}.xlsx', engine='openpyxl') as writer:
            for df, nome_aba in zip(pages_data, pages_names):
                df.to_excel(writer, sheet_name=nome_aba, index=False)
    else:
        print(f"No valid data available to write to Excel for table number {table_number}")


Requesting https://apisidra.ibge.gov.br/values/t/6588//n1/1/n2/all/n3/22/v/109//p/201201-202403/C48/all/f/n/d/4/h/y...
Requesting https://apisidra.ibge.gov.br/values/t/6588//n1/1/n2/all/n3/22/v/216//p/201201-202403/C48/all/f/n/d/4/h/y...
Requesting https://apisidra.ibge.gov.br/values/t/6588//n1/1/n2/all/n3/22/v/35//p/201201-202403/C48/all/f/n/d/4/h/y...
Requesting https://apisidra.ibge.gov.br/values/t/6588//n1/1/n2/all/n3/22/v/36//p/201201-202403/C48/all/f/n/d/4/h/y...
Requesting https://apisidra.ibge.gov.br/values/t/7832//n1/1/n2/all/n3/22/v/109//p/201201-202403/C49/all/f/n/d/4/h/y...
Requesting https://apisidra.ibge.gov.br/values/t/7832//n1/1/n2/all/n3/22/v/216//p/201201-202403/C49/all/f/n/d/4/h/y...
Requesting https://apisidra.ibge.gov.br/values/t/7832//n1/1/n2/all/n3/22/v/35//p/201201-202403/C49/all/f/n/d/4/h/y...
Requesting https://apisidra.ibge.gov.br/values/t/7832//n1/1/n2/all/n3/22/v/36//p/201201-202403/C49/all/f/n/d/4/h/y...
Requesting https://apisidra.ibge.gov.br/values/t/787

In [ ]:
dm = DirectoryManager(local_dir, final_dir)
dm.organize_files()